In [164]:
import spacy
nlp = spacy.load("en_core_web_lg")

book = open('text.txt', encoding='utf-8')
text = book.read()

# book.close()

doc = nlp(text)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)



Noun phrases: ['\n\nThis book', 'events', 'The incubation period', 'the viruses', 'this book', 'less than twenty-four days', 'No one', 'who', 'any', 'the viruses', 'who', 'contact', 'them', 'the viruses', 'the incubation\nperiod', 'None', 'the living people', 'this book', 'a\ncontagious disease', 'The viruses', 'more than\nten days', 'the viruses', 'special procedures', 'laboratory equipment', 'none', 'the locations', 'Reston', 'D.C.', 'this book', 'The second angel', 'his bowl', 'the sea', 'it', 'the blood', 'a dead man', '- APOCALYPSE\n\n\n\nPART', 'THE SHADOW', 'MOUNT ELGON', 'SOMETHING', 'THE FOREST\n1980 NEW YEAR S DAY\n\n\nCHARLES MONET', 'a loner', 'He', 'a Frenchman', 'who', 'himself', 'a little wooden bungalow', 'the private lands', 'Nzoia Sugar Factory', 'a\nplantation', 'western Kenya', 'what', 'the Nzoia River', 'sight', 'Mount Elgon', 'a huge, solitary extinct volcano', 'that', 'a height', 'fourteen thousand feet', 'the edge', 'the Rift Valley', "Monet's history", 'so many

In [165]:
from difflib import SequenceMatcher
import difflib

people = []
for ent in doc.ents:
    if ent.label_ == "PERSON" and ent.text[0].isupper() and ent.text[0].isalpha() and not ent.text.isupper():
        if ent.text not in people and """\n""" not in ent.text and "'s" not in ent.text:
            people.append(ent.text.strip())
        elif ent.text not in people and """\n""" in ent.text and "'s" not in ent.text:
            n_word = ent.text.split("""\n""")
            people.append(''.join(n_word).strip())
        elif ent.text not in people and """\n""" not in ent.text and "'s" in ent.text:
            s_word = ent.text.split("'s")
            people.append(s_word[0].strip())


compound_people = []
for person in people:
    if person.count(" ")>=1:
        compound_people.append(person)
# print(compound_people)
# 


unique_names = []
new_list = []
final_list = []
n = 30
cutoff = 0.9
for to_compare in compound_people:
    close_match = difflib.get_close_matches(to_compare, compound_people,n,cutoff)
    unique_names.append(close_match)
# print(unique_names)
# # Permutationer tas bort här:
# output = set(map(lambda x: tuple(sorted(x)),unique_names))

#Denna tar också bort permutationer!
result = []
for i in unique_names:
    i.sort()
    result.append(i)
output = []
for i in result:
    if i not in output:
        output.append(i)
output = list(map(tuple, output))
 
print(tuple(output))
 
# printing output
# print(output)

    # print(difflib.get_close_matches(to_compare, compound_people,n,cutoff))




(('Charles Monet', 'Charles Monet', 'Charles Mont'), ('Endebess Bluff',), ('Shem Musoke',), ('Antonia Bagshawe',), ('Imre Lofler',), ('Daniel arap Moi',), ('Beryl Markham',), ('Klaus F.',), ('David Silverstein',), ('Nancy Jaax',), ('Gerald Jaax',), ('Jerry Jaax', 'Jerry Jaax'), ('Fort Detrick', 'Lort Detrick'), ('Richard M. Nixon',), ('Eugene Johnson',), ('NANCY Jaax',), ('Gene Johnson', 'Gene Johnson', 'Gene Johnson', 'Gene Johnson'), ('Grateful Dead',), ('NANCY Jaax VOLUNTEERED',), ('Anthony Johnson',), ('Tony Johnson', 'Tony Johnson', 'Tony Johnson'), ('Ziploc zipper',), ('Mayinga N.',), ('N. Jaax',), ('Sister E.R.',), ('Sister M.E.',), ('Mobutu Sese Seko',), ('Karl Johnson', 'Karl M.Johnson'), ('Patricia Webb', 'Patricia Webb -'), ('Patrica Webb', 'Patricia Webb'), ('Frederick A. Murphy', 'Frederick A.Murphy'), ('Patrica Webb', 'Patricia Webb', 'Patricia Webb -'), ('Joel Breman',), ('Margaretha Isaacson',), ('Peter Cardinal', 'Peter Cardinal', 'Peter Cardinal', 'Peter Cardinal', 'P

In [166]:
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

#https://medium.com/analytics-vidhya/text-summarization-using-spacy-ca4867c6b744

# len(list(doc.sents))

#Filtering tokens
keyword = []
stopword = list(STOP_WORDS)
pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']
for token in doc:
    if token.text in stopword or token.text in punctuation:
        continue
    if token.pos_ in pos_tag:
        keyword.append(token.text)
        
freq_word = Counter(keyword)

#Normalization
max_freq = Counter(keyword).most_common(1)[0][1]
for word in freq_word.keys():
    freq_word[word] = (freq_word[word]/max_freq)
freq_word.most_common(20)

print(freq_word.most_common(20))

#Weighing sentences
sent_strength={}
for sent in doc.sents:
    for word in sent:
        if word.text in freq_word.keys():
            if word in sent_strength.keys():
                sent_strength[sent] += freq_word[word.text]
            else:
                sent_strength[sent]=freq_word[word.text]
# print(sent_strength)

#Summarization
summarized_sentences = nlargest(5,sent_strength,key=sent_strength.get)

final_sentences = [w.text for w in summarized_sentences]
summary = ' '.join(final_sentences)
# print(summary)
def generate_title(text):

    doc = nlp(text)

    # Extract important information or entities
    entities = [ent.text for ent in doc.ents]

    # Generate a title based on the extracted information
    title = f" ".join(entities) + " (the book)" if entities else "Untitled Document"

    return title

# Example usage

title = generate_title(summary)
print("Generated Title:", title)

[('virus', 1.0), ('Ebola', 0.9113300492610837), ('monkey', 0.9039408866995073), ('monkeys', 0.7881773399014779), ('blood', 0.7266009852216748), ('said', 0.5788177339901478), ('room', 0.4605911330049261), ('people', 0.41379310344827586), ('Nancy', 0.4039408866995074), ('air', 0.4014778325123153), ('suit', 0.3842364532019704), ('went', 0.3669950738916256), ('going', 0.3645320197044335), ('space', 0.3522167487684729), ('house', 0.3497536945812808), ('Jerry', 0.3448275862068966), ('time', 0.33004926108374383), ('building', 0.3251231527093596), ('Johnson', 0.31773399014778325), ('hot', 0.3078817733990148)]
Generated Title: a hundred million Silverstein Musoke American Kenya South Africans Musoke Marburg (the book)


In [167]:


# 
# for names in new_list:
#     close_match = difflib.get_close_matches(names, compound_people,n,cutoff)
#     final_list.append(close_match)
#     
# print(final_list)

# 
# for to_compare in compound_people:
#     for to_match in compound_people:
#         
#         # compare 
#         matches = SequenceMatcher(None, to_compare, to_match).get_matching_blocks() 
#   
#         for ele in matches: 
#             print(to_compare[ele.a:ele.a + ele.size])
#         
#         # temp = SequenceMatcher(None,to_compare ,to_match).ratio()
#         # 
#         # print(to_match + ' and ' + to_compare + ': ' + str(temp) + '\n')

#             
# 
# print(unique_names)
        
        # if temp > 0.9 and to_compare not in unique_names and to_match not in unique_names:
        #     tc_split = to_compare.split()
        #     tm_split = to_match.split()
        #     for part in tc_split:
        #         if part in to_match:
        #             if len(to_match)> len(to_compare):
        #                   unique_names.append(to_match)
        #             elif len(to_match)< len(to_compare):
        #                     unique_names.append(to_compare)
        #         elif to_match in to_compare:
        #             if len(to_match)> len(to_compare):
        #                   unique_names.append(to_match)
        #             elif len(to_match)< len(to_compare):
        #                     unique_names.append(to_compare)
        #         else:
        #             unique_names.append(to_compare)
        #     for part in tm_split:
        #         if part in to_match:
        #             if len(to_match)> len(to_compare) and to_match not in unique_names:
        #                   unique_names.append(to_match)
        #             elif len(to_match)< len(to_compare) and to_compare not in unique_names:
        #                     unique_names.append(to_compare)
        #         elif to_match in to_compare:
        #             if len(to_match)> len(to_compare) and to_match not in unique_names:
        #                   unique_names.append(to_match)
        #             elif len(to_match)< len(to_compare) and to_compare not in unique_names:
        #                     unique_names.append(to_compare)
        #         else:
        #             unique_names.append(to_compare)
# print(unique_names)


#Find similar words
# Prefer words that are separated over compound word

In [168]:
    # global unique_characters
    for entity in doc.ents:
        people = []
        for ent in doc.ents:
            if ent.label_ == "PERSON" and ent.text[0].isupper() and ent.text[0].isalpha() and not ent.text.isupper():
                if ent.text not in people and """\n""" not in ent.text and "'s" not in ent.text:
                    people.append(ent.text.strip())
                elif ent.text not in people and """\n""" in ent.text and "'s" not in ent.text:
                    n_word = ent.text.split("""\n""")
                    people.append(''.join(n_word).strip())
                elif ent.text not in people and """\n""" not in ent.text and "'s" in ent.text:
                    s_word = ent.text.split("'s")
                    people.append(s_word[0].strip())
                    
    compound_people = []
    for person in people:
        if person.count(" ") >= 1:
            compound_people.append(person)
    # print(compound_people)

    unique_names = []
    n = 30
    cutoff = 0.9
    for to_compare in compound_people:
        close_match = difflib.get_close_matches(to_compare, compound_people, n, cutoff)
        unique_names.append(close_match)
    print(unique_names)


        # Permutationer tas bort här:
    result = []
    for i in unique_names:
        i.sort()
        result.append(i)
        output = []
        for i in result:
            if i not in output:
                output.append(i)
    output = list(map(tuple, output))
    print(output)

        
 


[['Charles Monet', 'Charles Monet', 'Charles Mont'], ['Endebess Bluff'], ['Charles Mont', 'Charles Monet', 'Charles Monet'], ['Shem Musoke'], ['Antonia Bagshawe'], ['Imre Lofler'], ['Daniel arap Moi'], ['Beryl Markham'], ['Klaus F.'], ['David Silverstein'], ['Charles Monet', 'Charles Monet', 'Charles Mont'], ['Nancy Jaax'], ['Gerald Jaax'], ['Jerry Jaax', 'Jerry Jaax'], ['Fort Detrick', 'Lort Detrick'], ['Richard M. Nixon'], ['Eugene Johnson'], ['NANCY Jaax'], ['Gene Johnson', 'Gene Johnson', 'Gene Johnson', 'Gene Johnson'], ['Grateful Dead'], ['Gene Johnson', 'Gene Johnson', 'Gene Johnson', 'Gene Johnson'], ['NANCY Jaax VOLUNTEERED'], ['Anthony Johnson'], ['Tony Johnson', 'Tony Johnson', 'Tony Johnson'], ['Tony Johnson', 'Tony Johnson', 'Tony Johnson'], ['Ziploc zipper'], ['Mayinga N.'], ['Tony Johnson', 'Tony Johnson', 'Tony Johnson'], ['N. Jaax'], ['Gene Johnson', 'Gene Johnson', 'Gene Johnson', 'Gene Johnson'], ['Sister E.R.'], ['Sister M.E.'], ['Mobutu Sese Seko'], ['Karl M.Johnso